<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/NIFTY_LOW_backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%pip install pya3
%pip install nsetools

In [14]:
#NIFTY
from pya3 import *


def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

# Example usage:
user_id, api_key = get_user_credentials()
alice = Aliceblue(user_id=user_id, api_key=api_key)
# Now you can use the 'alice' object in the rest of your code.



#you will get ok after successfully login in alice blue using the api
alice.get_session_id()

from datetime import datetime, timedelta
from nsetools import Nse
import holidays
import datetime



# Create an instance of Nse
nse = Nse()

# Get the list of market holidays for the year
market_holidays = holidays.India(years=datetime.datetime.now().year)

# Add Mahashivratri as a market holiday on 08-03-24
market_holidays.append(datetime.date(2024, 3, 8))

# Get the current date
current_date = datetime.datetime.now()

# Find the previous trading day
previous_trading_day = current_date - timedelta(days=1)
while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
    previous_trading_day -= timedelta(days=1)


yesterday_date = previous_trading_day.date()

instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY31OCT24F')
print(instrument)

from_datetime = datetime.datetime.combine(yesterday_date, datetime.time.min)  # From yesterday
to_datetime = datetime.datetime.combine(yesterday_date, datetime.time.max)  # To yesterday
interval = "D"  # ["1", "D"]
historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
print(historical_data)

last_close_price = historical_data['close'].iloc[-1]
print("Closing Price of the Last Minute:", last_close_price)

# Store the closing price for later use
closeprice = last_close_price

# Now you can use 'stored_close_price' in your subsequent code
print("Closing Price:", closeprice)

# Function to check if a given date is a holiday
def is_holiday(date):
    india_holidays = holidays.CountryHoliday('IND', prov='MH')
    return date in india_holidays

# Find the next Thursday from the current date
def next_thursday():
    today = datetime.date.today()
    days_ahead = 3 - today.weekday()  # 3 corresponds to Thursday
    if days_ahead <= 2:  # Target day has already happened this week
        days_ahead += 7
    return today + datetime.timedelta(days=days_ahead)

# Assuming you have the closing price stored in 'last_close_price'
closing_price = closeprice  # Replace this with the actual closing price from your data

# Calculate the nearest strike prices
ce_strike_price = int(closing_price / 50) * 50
pe_strike_price = ce_strike_price + 50

# Get the next Thursday for expiry date
expiry_date = next_thursday()

# Construct option symbols with expiry date and strike prices
symbol_ce = f'NIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
symbol_pe = f'NIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'

print("CE Option Symbol:", symbol_ce)
print("PE Option Symbol:", symbol_pe)



# Define the instrument symbol_pe
instrument1 = alice.get_instrument_by_symbol('NFO', symbol_ce)

# Get historical data
historical_data = alice.get_historical(instrument1, from_datetime, to_datetime, interval)
print(historical_data)

# Assuming 'data' is a DataFrame
high_values = historical_data['low'].tolist()
print(high_values)

highest_high_ce = historical_data['low'].max()
print("Highest High Value:", highest_high_ce)

highest_high_ce = round(highest_high_ce)
highest_high_ce = float(highest_high_ce)
print("Rounded Highest High Value:", highest_high_ce)


# Define the instrument symbol_pe
instrument2 = alice.get_instrument_by_symbol('NFO', symbol_pe)


# Get historical data
historical_data = alice.get_historical(instrument2, from_datetime, to_datetime, interval)
print(historical_data)

# Assuming 'data' is a DataFrame
high_values = historical_data['low'].tolist()
print(high_values)

highest_high_pe = historical_data['low'].max()
print("Highest High Value:", highest_high_pe)

highest_high_pe = round(highest_high_pe)
highest_high_pe = float(highest_high_pe)
print("Rounded Highest High Value:", highest_high_pe)


#Placing order for ce
# Define a variable name as a string
CE_high = "highest_high_ce"

# Retrieve the variable value using globals()
price_value = globals()[CE_high]

print(
   alice.place_order(transaction_type = TransactionType.Buy,
                     instrument = alice.get_instrument_by_symbol('NFO', symbol_ce),
                     quantity = 25,
                     order_type = OrderType.StopLossLimit,
                     product_type = ProductType.BracketOrder,
                     price = price_value,  # Pass the retrieved value
                     trigger_price = price_value,
                     stop_loss =  100.0,
                     square_off = 10.0,
                     trailing_sl = 0.0,
                     is_amo = True,
                     order_tag='order1')
)



#placing order for pe
# Define a variable name as a string
PE_high = "highest_high_pe"

# Retrieve the variable value using globals()
price_value = globals()[PE_high]

print(
   alice.place_order(transaction_type = TransactionType.Buy,
                     instrument = alice.get_instrument_by_symbol('NFO', symbol_pe),
                     quantity = 25,
                     order_type = OrderType.StopLossLimit,
                     product_type = ProductType.BracketOrder,
                     price = price_value,  # Pass the retrieved value
                     trigger_price = price_value,
                     stop_loss =  100.0,
                     square_off = 10.0,
                     trailing_sl = 0.0,
                     is_amo = True,
                     order_tag='order1')
)


Enter your user ID: 1141826
Enter your API key: COuxpq3aPZ5WHZdxivMuRPM9904hNcV9xeJMX3AlhCbvNtktS7c4ZtTlpSm2iMEoDesYl4z7ASCyJ6m87sX7dpU4vM6CVz3rMlAAJkF7t6dzl8h7hpW9OzI1oIHDgTCV
Instrument(exchange='NFO', token=35382, symbol='NIFTY', name='NIFTY31OCT24F', expiry='', lot_size=25)
              datetime     open     high      low    close    volume
0  2024-10-17 00:00:00  25068.7  25074.9  24815.1  24843.8  193005.0
Closing Price of the Last Minute: 24843.8
Closing Price: 24843.8
CE Option Symbol: NIFTY24Oct24C24800
PE Option Symbol: NIFTY24Oct24P24850
              datetime   open    high     low  close    volume
0  2024-10-17 00:00:00  302.0  323.35  147.85  163.3  694965.0
[147.85]
Highest High Value: 147.85
Rounded Highest High Value: 148.0
              datetime  open   high    low   close    volume
0  2024-10-17 00:00:00  98.8  199.5  86.45  182.45  224760.0
[86.45]
Highest High Value: 86.45
Rounded Highest High Value: 86.0
{'stat': 'Ok', 'NOrdNo': '24101800265663'}
{'stat': 'Ok', '

In [24]:
from_datetime = datetime.datetime.combine(current_date, time(9, 15))
to_datetime = datetime.datetime.combine(current_date, time(15, 30))
interval = "1"  # ["1", "D"]

historical_data = alice.get_historical(instrument1, from_datetime, to_datetime, interval)
print(historical_data)


                datetime    open    high     low   close    volume
0    2024-10-18 09:15:59  115.00  139.60  104.20  105.50  620925.0
1    2024-10-18 09:16:59  104.35  104.75   99.05  101.05  541275.0
2    2024-10-18 09:17:59  101.50  102.85   98.65   99.65  387075.0
3    2024-10-18 09:18:59  100.00  100.00   93.00   96.40  448425.0
4    2024-10-18 09:19:59   96.05   99.10   94.95   97.35  353225.0
..                   ...     ...     ...     ...     ...       ...
370  2024-10-18 15:25:59  196.10  198.35  196.05  197.85   99600.0
371  2024-10-18 15:26:59  198.65  199.90  196.40  196.75   62725.0
372  2024-10-18 15:27:59  196.95  199.85  196.00  199.70  113250.0
373  2024-10-18 15:28:59  199.55  200.00  198.00  198.90   91100.0
374  2024-10-18 15:29:59  199.00  199.60  195.60  197.95  156675.0

[375 rows x 6 columns]


In [22]:
print(highest_high_ce)
print(highest_high_pe)

148.0
86.0


In [23]:
high_ce = highest_high_ce

In [28]:
import pandas as pd

def backtest(historical_data, high_ce, target_points=50, stop_loss_points=10):
    # Set the target and stop-loss levels
    target_price = high_ce + target_points
    stop_loss_price = high_ce - stop_loss_points

    # Initialize variables to track trade status
    trade_open = False
    entry_time = None
    exit_time = None
    result = None  # 'Win' or 'Loss'

    # Iterate through the historical data
    for index, row in historical_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            # Check if we should enter the trade (if the buying price is touched)
            if low <= high_ce <= high:
                # Buy order is triggered
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened at {entry_time} with entry price {high_ce}")

        if trade_open:
            # Check if stop-loss or target is hit
            if low <= stop_loss_price:
                # Stop-loss is hit
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit at {exit_time} with price {stop_loss_price}")
                break
            elif high >= target_price:
                # Target is hit
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit at {exit_time} with price {target_price}")
                break

    # If trade is still open by the end of the day, log the result
    if trade_open and result is None:
        result = 'Open'  # Trade is still open

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result
    }

# Ensure 'datetime' column is in datetime format
historical_data['datetime'] = pd.to_datetime(historical_data['datetime'])

# Set your buying price
high_ce

# Call the backtesting function
backtest_result = backtest(historical_data, high_ce)

# Output the result
print(backtest_result)


Trade opened at 2024-10-18 10:33:59 with entry price 148
Stop-loss hit at 2024-10-18 10:39:59 with price 138
{'Entry Time': Timestamp('2024-10-18 10:33:59'), 'Exit Time': Timestamp('2024-10-18 10:39:59'), 'Result': 'Loss'}
